In [2]:
import os
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import matplotlib.pylab as plt

%matplotlib inline

from urllib.request import urlopen
from pathlib import Path
from collections import defaultdict
from random import choice
from random import sample

In [3]:
red_lit = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_LIT.txt')
red_apms = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_AP-MS.txt')
red_bin = nx.read_edgelist('./Material_de_interes/tc02Data/yeast_Y2H.txt')

In [4]:
escenciales = open('./Datos/Essential_ORFs_paperHe2.txt')
lista_escenciales = []

for line in escenciales:
    lista_escenciales.append(line.rstrip('\n'))

In [5]:
def nodos_esenciales(red, lista_atr = lista_escenciales, atributo = 'Esencial'):
    
    dict_vacio = dict()
    nx.set_node_attributes(red, dict_vacio.fromkeys(list(red.nodes()),0), atributo )
    
    for nodo in lista_atr:
        try:
            red.nodes[nodo][atributo] = 1
        except KeyError:
            continue
            
    return None

In [6]:
nodos_esenciales(red_lit)
nodos_esenciales(red_bin)
nodos_esenciales(red_apms)

In [7]:
def nodosEsenciales(red, esenciales = lista_escenciales, atributo = 'Esencial'):
    red_aux = red.copy()
    red_aux.remove_nodes_from(list(nodosNoEsenciales(red).nodes()))
    return red_aux.nodes()

In [8]:
def nodosNoEsenciales(red, esenciales = lista_escenciales):
    red_aux = red.copy()
    red_aux.remove_nodes_from(esenciales)
    return red_aux

In [9]:
def sacarEsenciales(red):
    red_aux = red.copy()
    GC = max(nx.connected_component_subgraphs(red_aux), key=len)
    n = GC.number_of_nodes()
    red_aux.remove_nodes_from(list(nodosEsenciales(red)))
    GC = max(nx.connected_component_subgraphs(red_aux), key=len)
    m = GC.number_of_nodes()
    return m/n

In [66]:
sacarEsenciales(red_lit)

0.281121187139324

In [10]:
def sampleNodos(red,dic1,dic2,k,q,dif1=0):
    if dif1 == 0:
        dif1 = dic1[k][1]
    try:
        dif2 = dic1[q][0] - dif1
        sample_nodos = sample(dic2[q],dif1)
        red.remove_nodes_from(sample_nodos)
        if dif2 <= 0:
            return None
        else:
            sampleNodos(red,dic1,dic2,k,q-1,dif1=dif2)
            
    except:
        sampleNodos(red,dic1,dic2,k,q-1,dif1=0)

In [11]:
def randomNoEsenciales(red):
    red_aux = red.copy()
    n = red.number_of_nodes()
    
    degs = sorted(dict(red.degree()).values(), reverse=True)
    nodos = dict(red.nodes(data=True))
    grados = {d: [0,0] for d in degs}
    noEsencXGrado = defaultdict(list)
    for nodo, deg in red.degree():
        grados[deg][0] += nodos[nodo]['Esencial']
        grados[deg][1] += 1-nodos[nodo]['Esencial']
        if (not nodos[nodo]['Esencial']):
            noEsencXGrado[deg].append(nodo)
    grad = sorted(list(grados.keys()))
    for grado in grad:
        sampleNodos(red_aux,grados,noEsencXGrado,grado,grado,dif1=0)
    return red_aux.number_of_nodes()/n

In [ ]:
randomNoEsenciales(red_lit)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



In [14]:
red_lit.number_of_nodes()

1536